In [1]:
import pickle
import pandas as pd
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import random
import itertools
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(42)
random.seed(42)

In [2]:
# path = '/Users/thomas/Downloads/nturgb+d_skeletons'
path = 'D:\\Datasets\\Motion Privacy\\NTU RGB+D 120\\Skeleton Data'
X_path = 'data/X.pkl'

## Data organization

In [3]:
def load_files():
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    return files_
files_ = None

In [4]:
# Attempt to load X and Y from pickle before generating them
X = {}
try:
    print('Attempting to load X from pickle')
    with open(X_path, 'rb') as f:
        X = pickle.load(f)
    print('X loaded from pickle')
except:
    print('Could not load X and Y, generating them now')
    
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    # Generate X and Y
    for file_ in tqdm(files_, desc='Files Parsed', position=0):
        try:
            file = join(path, file_['file'])
            data = open(file, 'r')
            lines = data.readlines()
            frames_count = int(lines.pop(0).replace('\n', ''))
            file_['frames'] = frames_count
        except UnicodeDecodeError: # .DS_Store file
            print('UnicodeDecodeError: ', file)
            continue

        # Add filename as key to X
        X[file_['file']] = []

        # Skip file if 2 actors
        if lines[0].replace('\n', '') != '1': continue

        for f in tqdm(range(frames_count), desc='Frames Parsed', position=1, leave=False):
            try:
                # Get actor count
                actors = int(lines.pop(0).replace('\n', ''))
            
                # Get actor info
                t = lines.pop(0)

                # Get joint count
                joint_count = int(lines.pop(0).replace('\n', ''))

                # Get joint info
                d = []
                for j in range(joint_count):
                    joint = lines.pop(0).replace('\n', '').split(' ')
                    d.extend(joint[0:3])

                # Skip if not 25 joints
                if len(d) != 75: continue

                # Convert to numpy array
                d = np.array(d)

                # Append to X and Y
                X[file_['file']].append(d)
            except:
                break
        
        # Convert to numpy array
        X[file_['file']] = np.array(X[file_['file']], dtype=np.float16)

        # Pad X size to 300 frames (300 is max frames in dataset)
        X[file_['file']] = np.pad(X[file_['file']], ((0, 300-X[file_['file']].shape[0]), (0, 0)), 'constant')


    print('X Generated, saving to pickle...')

    # Save the data
    with open(X_path, 'wb') as f:
        pickle.dump(X, f)

    print('X Saved to pickle')


Attempting to load X from pickle
X loaded from pickle


In [5]:
for file in X:
    # Clip X to 50 frames
    X[file] = X[file][:50]

In [6]:
same_samples_per_actor = 100
diff_samples_per_actor = 100


def data_generator_per_actor(X, same_samples_per_actor=1000, diff_samples_per_actor=1000, train=True):
    actor_data = {}
    for file in X:
        actor = int(file[9:12])
        action = int(file[17:20])

        if train: 
            if action > 60:
                continue
        else:
            if action <= 60:
                continue

        if actor not in actor_data:
            actor_data[actor] = []
        if len(X[file]) == 0:
            continue
        actor_data[actor].append(X[file])

    actor_keys = list(actor_data.keys())

    samples = []
    for actor in tqdm(actor_keys):
        for _ in range(same_samples_per_actor):
            same_video1 = random.choice(actor_data[actor])
            same_video2 = random.choice(actor_data[actor])
            samples.append(((tf.convert_to_tensor(same_video1, dtype=tf.float32), tf.convert_to_tensor(same_video2, dtype=tf.float32)), tf.constant([1.0], dtype=tf.float32)))

        for _ in range(diff_samples_per_actor):
            while True:
                diff_actor = random.choice(actor_keys)
                if diff_actor != actor:
                    break
            same_video1 = random.choice(actor_data[actor])
            diff_video = random.choice(actor_data[diff_actor])
            samples.append(((tf.convert_to_tensor(same_video1, dtype=tf.float32), tf.convert_to_tensor(diff_video, dtype=tf.float32)), tf.constant([0.0], dtype=tf.float32)))

        random.shuffle(samples)

    return samples


train_gen = data_generator_per_actor(X, same_samples_per_actor, diff_samples_per_actor, train=True)
val_gen = data_generator_per_actor(X, same_samples_per_actor, diff_samples_per_actor, train=False)
test_gen = data_generator_per_actor(X, same_samples_per_actor, diff_samples_per_actor, train=False)

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

In [7]:
class TransformerBlock(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()

        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training):
        attn_output, _ = self.mha(x, x, return_attention_scores=True)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

class SimpleTransformer(Model):
    def __init__(self, num_layers, d_model, num_heads, dff):
        super(SimpleTransformer, self).__init__()
        self.transformer_layers = [TransformerBlock(d_model, num_heads, dff) for _ in range(num_layers)]
        self.global_average_pooling = layers.GlobalAveragePooling1D()
        self.flatten = layers.Flatten()
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=True):
        x_a = inputs[0]
        x_b = inputs[1]
        for transformer_layer in self.transformer_layers:
            x_a = transformer_layer(x_a, training=training)
            x_b = transformer_layer(x_b, training=training)
        x_a = self.global_average_pooling(x_a)
        x_b = self.global_average_pooling(x_b)
        x = x_a*x_b
        x = self.flatten(x)
        x = self.output_layer(x)
        return x

# Example usage:
model = SimpleTransformer(num_layers=3, d_model=75, num_heads=5, dff=256)

In [8]:
batch_size = 64

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_signature=(
        (tf.TensorSpec(shape=(50, 75), dtype=tf.float32),
         tf.TensorSpec(shape=(50, 75), dtype=tf.float32)),
        tf.TensorSpec(shape=(1,), dtype=tf.float32)
    )
).batch(batch_size)

val_dataset = tf.data.Dataset.from_generator(
    lambda: val_gen,
    output_signature=(
        (tf.TensorSpec(shape=(50, 75), dtype=tf.float32),
         tf.TensorSpec(shape=(50, 75), dtype=tf.float32)),
        tf.TensorSpec(shape=(1,), dtype=tf.float32)
    )
).batch(batch_size)

# Create and compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=1000, validation_data=val_dataset)


Epoch 1/1000
125/125 [==============================] - 25s 161ms/step - loss: 0.7577 - accuracy: 0.5059 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 2/1000
125/125 [==============================] - 17s 137ms/step - loss: 0.6988 - accuracy: 0.4919 - val_loss: 0.6942 - val_accuracy: 0.5000
Epoch 3/1000
125/125 [==============================] - 14s 109ms/step - loss: 0.6972 - accuracy: 0.4959 - val_loss: 0.6958 - val_accuracy: 0.5000
Epoch 4/1000
125/125 [==============================] - 11s 91ms/step - loss: 0.6964 - accuracy: 0.4870 - val_loss: 0.6955 - val_accuracy: 0.5000
Epoch 5/1000
125/125 [==============================] - 14s 112ms/step - loss: 0.6957 - accuracy: 0.4885 - val_loss: 0.6948 - val_accuracy: 0.5000
Epoch 6/1000
125/125 [==============================] - 12s 96ms/step - loss: 0.6955 - accuracy: 0.4921 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 7/1000
125/125 [==============================] - 11s 89ms/step - loss: 0.6951 - accuracy: 0.4947 - val_loss: 0.69

KeyboardInterrupt: 